In [1]:
import pandas as pd
import os
import numpy as np
import re

pd.set_option("display.max_columns", 999)

In [2]:
os.getcwd()

'/Users/hayley/Documents/p4ds/patent_search/data_preprocess'

In [3]:
os.chdir('/Users/hayley/Documents/p4ds/patent_search')

In [ ]:
# sample_data = pd.read_csv('data_preprocess/data.csv')

In [ ]:
# sample_data.to_excel("data_preprocess/data.xlsx")

In [ ]:
# sample_data = pd.read_excel('data_preprocess/data.xlsx')

In [ ]:
# for col in sample_data.columns:
#     sample_data[col] = sample_data[col].str.replace('\n', '')

In [ ]:
# sample_data.to_excel("data_preprocess/data.xlsx")

# Preprocessing
- 1) problem with extracting reference (number) from texts -> some reference codes are not just numbers. it's number + alphabet e.g. 202a, 202b, or sometimes just uppercase alphabets e.g. T, SR~SZ
- 2) so I tried to first extract the reference codes from "부호에 대한 설명" column. (Because then I can find those specific reference codes inside of texts.) But extracting reference codes from "부호에 대한 설명" was difficult because currently all \<code\>: \<description\> string pairs are concatenated without \n and the parsing reference codes by just relying on regular expression was almost impossible. (some reference codes were uppercase alphabets, but including uppercase alphabets for codes caused problems.) So I asked Mooho if he could leave \n characters left for "부호에 대한 설명" section. He said yes, and I paused developing regex rules further.

In [ ]:
# sample_data = pd.read_csv('pdf_process/data.csv', index_col=0)
# sample_data.to_excel('pdf_process/data.xlsx')
sample_data = pd.read_excel('pdf_process/data.xlsx')

In [ ]:
def extract_ref(x):
    
    init_extract = set(re.findall(r"\(\d*[a-zA-Z]?\)|도 \d+", str(x)))
    if len(init_extract) > 0:
        return init_extract
    else:
        return np.nan

In [ ]:
select_columns = ['요약', '청구범위', '기술분야', '배경기술', '해결하려는과제', '과제의해결수단',
                 '발명의효과', '발명을실시하기위한구체적인내용']

In [ ]:
for col in select_columns:
    sample_data[f"{col}_ref"] = sample_data[col].apply(extract_ref)

In [ ]:
def extract_description_for_image(x):
    descs = re.findall(r'도 ?\d+[의은는][^\.]+\.', x)
    
    return dict(zip([f'도 {i}' for i in range(1, len(descs)+1)], descs))

sample_data['도면의간단한설명_dict'] = sample_data['도면의간단한설명'].apply(extract_description_for_image)

In [ ]:
def extract_description_for_code(x):
    ids = re.findall(r'([\d]+)[a-zA-Z]?,? ?[\da-zA-Z]*(?= ?[:;]+)', str(x)) # became ugly to take care of cases like '311b, 311c'
    ids = [x.strip() for x in ids]
    descs = re.findall(r'(?<=[:;])[\-0-9ㄱ-ㅎ가-힣a-zA-Z ]+', str(x))
    descs = [re.sub(r'([\d]+)[a-zA-Z]?$', '', x.strip()) for x in descs]
    descs = [x.strip() for x in descs]
    if len(descs)>0: # handling noise from data parsing ('도면' got attached to '부호의설명' column)
        descs[-1] = re.sub('도면.+', '', descs[-1]).strip()
        
    return_dict = dict(zip(ids, descs))
    
    initial_keys = list(return_dict.keys()) # if there're keys like '311b, 311c', split the key into two different keys and add the same value.
    for key in initial_keys:
        if ',' in key:
            value = return_dict[key]
            return_dict.pop(key)
            subkey_list = key.split(',')
            for subkey in subkey_list:
                return_dict[subkey.strip()] = value
    
    return return_dict

In [ ]:
sample_data.loc[:,'부호의설명_dict'] = sample_data['부호의설명'].apply(extract_description_for_code)

In [81]:
print(pytesseract.image_to_string(Image.open('data_preprocess/image/pdf_1020177009557/p11-53.png')))

# OCR

- Tesseract : 잘 못함
- 다른 오픈소스 : pyocr -> tesseract랑 똑같음 / calamari-ocr -> tensorflow 설치해야 되고 등등 maintain이 잘 안되는 패키지 인듯
- GCP vision api : 일단 ui 에서 테스트 해보고 copy json output 버튼 눌러가지고 json output 복사해와서 json output에서 텍스트만 파싱하는 것만 짬

In [ ]:
import pytesseract
from PIL import Image

## tesseract를 먼저 다운받고, 다운받은 경로를 넣어주어야 함.
## You need to first download tesseract & insert the path to the exe file below.
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/5.3.3/bin/tesseract'

In [47]:
# text image to string
from glob import glob

image_paths = glob('data_preprocess/image/*/*.png')

for img in image_paths:
    print(img)
    print(pytesseract.image_to_string(Image.open(img), lang='eng'))

data_preprocess/image/pdf_1020120156759/p8-6.png

data_preprocess/image/pdf_1020120156759/p7-31.png
A256
ART
129
A120
A158
123,
121

133

A35
437

10 130110 145: 147
140- 143


data_preprocess/image/pdf_1020120156759/p7-30.png
110 5 110

1 ~_-f


data_preprocess/image/pdf_1020120156759/p0-6.png

data_preprocess/image/pdf_1020120156759/p9-39.png
200

212
214a

222 224a 226 228

data_preprocess/image/pdf_1020120156759/p8-35.png
200

300

320
310


data_preprocess/image/pdf_1020180014052/p14-56.png

data_preprocess/image/pdf_1020180014052/p12-6.png

data_preprocess/image/pdf_1020180014052/p15-60.png

data_preprocess/image/pdf_1020180014052/p0-6.png

data_preprocess/image/pdf_1020180014052/p13-52.png

data_preprocess/image/pdf_1020180014052/p11-43.png

data_preprocess/image/pdf_1020180014052/p11-44.png

data_preprocess/image/pdf_1020180014052/p11-45.png

data_preprocess/image/pdf_1020177009557/p13-66.png
1

4

5

\


data_preprocess/image/pdf_1020177009557/p13-67.png
La

4 a, 3 i
cs ot “
Q

In [71]:
for i in json_output['fullTextAnnotation']['pages'][0]['blocks']:
    for j in i['paragraphs']:
        for k in j['words']:
            curr_word = ''
            for l in k['symbols']:
                curr_word += l['text']
            if curr_word.startswith('-'):
                curr_word = curr_word[1:]
            print(curr_word)

30
100
40
20
10


In [76]:
annotations = json_output['textAnnotations'][0]['description'].split('\n')
print(annotations)

['-30', '100', '40', '20', '-10']


In [78]:
annotations = json_output['textAnnotations'][0]['description'].split('\n')
print(annotations)

['15', '***************', '17']


In [77]:
json_output = {
  "cropHintsAnnotation": {
    "cropHints": [
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 510
            },
            {
              "x": 1155
            },
            {
              "x": 1155,
              "y": 805
            },
            {
              "x": 510,
              "y": 805
            }
          ]
        },
        "confidence": 0.5625001,
        "importanceFraction": 0.67390424
      },
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 291
            },
            {
              "x": 1097
            },
            {
              "x": 1097,
              "y": 805
            },
            {
              "x": 291,
              "y": 805
            }
          ]
        },
        "confidence": 0.5625,
        "importanceFraction": 0.84238034
      },
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 218
            },
            {
              "x": 1185
            },
            {
              "x": 1185,
              "y": 805
            },
            {
              "x": 218,
              "y": 805
            }
          ]
        },
        "confidence": 0.55645883,
        "importanceFraction": 1
      }
    ]
  },
  "fullTextAnnotation": {
    "pages": [
      {
        "blocks": [
          {
            "blockType": "TEXT",
            "boundingBox": {
              "vertices": [
                {
                  "x": 44,
                  "y": 493
                },
                {
                  "x": 86,
                  "y": 493
                },
                {
                  "x": 86,
                  "y": 522
                },
                {
                  "x": 44,
                  "y": 522
                }
              ]
            },
            "confidence": 0.98961794,
            "paragraphs": [
              {
                "boundingBox": {
                  "vertices": [
                    {
                      "x": 44,
                      "y": 493
                    },
                    {
                      "x": 86,
                      "y": 493
                    },
                    {
                      "x": 86,
                      "y": 522
                    },
                    {
                      "x": 44,
                      "y": 522
                    }
                  ]
                },
                "confidence": 0.98961794,
                "words": [
                  {
                    "boundingBox": {
                      "vertices": [
                        {
                          "x": 44,
                          "y": 493
                        },
                        {
                          "x": 86,
                          "y": 493
                        },
                        {
                          "x": 86,
                          "y": 522
                        },
                        {
                          "x": 44,
                          "y": 522
                        }
                      ]
                    },
                    "confidence": 0.98961794,
                    "symbols": [
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 44,
                              "y": 494
                            },
                            {
                              "x": 62,
                              "y": 494
                            },
                            {
                              "x": 62,
                              "y": 522
                            },
                            {
                              "x": 44,
                              "y": 522
                            }
                          ]
                        },
                        "confidence": 0.98852473,
                        "text": "1"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 64,
                              "y": 493
                            },
                            {
                              "x": 86,
                              "y": 493
                            },
                            {
                              "x": 86,
                              "y": 521
                            },
                            {
                              "x": 64,
                              "y": 521
                            }
                          ]
                        },
                        "confidence": 0.9907112,
                        "property": {
                          "detectedBreak": {
                            "type": "LINE_BREAK"
                          }
                        },
                        "text": "5"
                      }
                    ]
                  }
                ]
              }
            ]
          },
          {
            "blockType": "TEXT",
            "boundingBox": {
              "vertices": [
                {
                  "x": 475,
                  "y": 504
                },
                {
                  "x": 102,
                  "y": 504
                },
                {
                  "x": 102,
                  "y": 492
                },
                {
                  "x": 475,
                  "y": 492
                }
              ]
            },
            "confidence": 0.33337253,
            "paragraphs": [
              {
                "boundingBox": {
                  "vertices": [
                    {
                      "x": 475,
                      "y": 504
                    },
                    {
                      "x": 102,
                      "y": 504
                    },
                    {
                      "x": 102,
                      "y": 492
                    },
                    {
                      "x": 475,
                      "y": 492
                    }
                  ]
                },
                "confidence": 0.33337253,
                "words": [
                  {
                    "boundingBox": {
                      "vertices": [
                        {
                          "x": 475,
                          "y": 504
                        },
                        {
                          "x": 102,
                          "y": 504
                        },
                        {
                          "x": 102,
                          "y": 492
                        },
                        {
                          "x": 475,
                          "y": 492
                        }
                      ]
                    },
                    "confidence": 0.33337253,
                    "symbols": [
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 475,
                              "y": 504
                            },
                            {
                              "x": 465,
                              "y": 504
                            },
                            {
                              "x": 465,
                              "y": 493
                            },
                            {
                              "x": 475,
                              "y": 493
                            }
                          ]
                        },
                        "confidence": 0.44515774,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 466,
                              "y": 503
                            },
                            {
                              "x": 455,
                              "y": 503
                            },
                            {
                              "x": 455,
                              "y": 492
                            },
                            {
                              "x": 466,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.36208847,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 452,
                              "y": 503
                            },
                            {
                              "x": 442,
                              "y": 503
                            },
                            {
                              "x": 442,
                              "y": 492
                            },
                            {
                              "x": 452,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.30566624,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 440,
                              "y": 503
                            },
                            {
                              "x": 430,
                              "y": 503
                            },
                            {
                              "x": 430,
                              "y": 492
                            },
                            {
                              "x": 440,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.26879194,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 428,
                              "y": 503
                            },
                            {
                              "x": 418,
                              "y": 503
                            },
                            {
                              "x": 418,
                              "y": 492
                            },
                            {
                              "x": 428,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.2546715,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 403,
                              "y": 503
                            },
                            {
                              "x": 393,
                              "y": 503
                            },
                            {
                              "x": 393,
                              "y": 492
                            },
                            {
                              "x": 403,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.2709255,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 326,
                              "y": 503
                            },
                            {
                              "x": 315,
                              "y": 503
                            },
                            {
                              "x": 315,
                              "y": 492
                            },
                            {
                              "x": 326,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.2530431,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 252,
                              "y": 503
                            },
                            {
                              "x": 241,
                              "y": 503
                            },
                            {
                              "x": 241,
                              "y": 492
                            },
                            {
                              "x": 252,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.27793998,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 188,
                              "y": 503
                            },
                            {
                              "x": 176,
                              "y": 503
                            },
                            {
                              "x": 176,
                              "y": 492
                            },
                            {
                              "x": 188,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.29446027,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 174,
                              "y": 503
                            },
                            {
                              "x": 163,
                              "y": 503
                            },
                            {
                              "x": 163,
                              "y": 492
                            },
                            {
                              "x": 174,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.34936973,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 161,
                              "y": 503
                            },
                            {
                              "x": 149,
                              "y": 503
                            },
                            {
                              "x": 149,
                              "y": 492
                            },
                            {
                              "x": 161,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.28837347,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 148,
                              "y": 503
                            },
                            {
                              "x": 134,
                              "y": 503
                            },
                            {
                              "x": 134,
                              "y": 492
                            },
                            {
                              "x": 148,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.25042403,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 137,
                              "y": 503
                            },
                            {
                              "x": 123,
                              "y": 503
                            },
                            {
                              "x": 123,
                              "y": 492
                            },
                            {
                              "x": 137,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.2917209,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 123,
                              "y": 503
                            },
                            {
                              "x": 112,
                              "y": 503
                            },
                            {
                              "x": 112,
                              "y": 492
                            },
                            {
                              "x": 123,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.47531936,
                        "text": "*"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 113,
                              "y": 503
                            },
                            {
                              "x": 102,
                              "y": 503
                            },
                            {
                              "x": 102,
                              "y": 492
                            },
                            {
                              "x": 113,
                              "y": 492
                            }
                          ]
                        },
                        "confidence": 0.6126359,
                        "property": {
                          "detectedBreak": {
                            "type": "LINE_BREAK"
                          }
                        },
                        "text": "*"
                      }
                    ]
                  }
                ]
              }
            ]
          },
          {
            "blockType": "TEXT",
            "boundingBox": {
              "vertices": [
                {
                  "x": 1365,
                  "y": 208
                },
                {
                  "x": 1412,
                  "y": 208
                },
                {
                  "x": 1412,
                  "y": 241
                },
                {
                  "x": 1365,
                  "y": 241
                }
              ]
            },
            "confidence": 0.99004245,
            "paragraphs": [
              {
                "boundingBox": {
                  "vertices": [
                    {
                      "x": 1365,
                      "y": 208
                    },
                    {
                      "x": 1412,
                      "y": 208
                    },
                    {
                      "x": 1412,
                      "y": 241
                    },
                    {
                      "x": 1365,
                      "y": 241
                    }
                  ]
                },
                "confidence": 0.99004245,
                "words": [
                  {
                    "boundingBox": {
                      "vertices": [
                        {
                          "x": 1365,
                          "y": 208
                        },
                        {
                          "x": 1412,
                          "y": 208
                        },
                        {
                          "x": 1412,
                          "y": 241
                        },
                        {
                          "x": 1365,
                          "y": 241
                        }
                      ]
                    },
                    "confidence": 0.99004245,
                    "symbols": [
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 1365,
                              "y": 209
                            },
                            {
                              "x": 1389,
                              "y": 209
                            },
                            {
                              "x": 1389,
                              "y": 241
                            },
                            {
                              "x": 1365,
                              "y": 241
                            }
                          ]
                        },
                        "confidence": 0.9902449,
                        "text": "1"
                      },
                      {
                        "boundingBox": {
                          "vertices": [
                            {
                              "x": 1387,
                              "y": 208
                            },
                            {
                              "x": 1412,
                              "y": 208
                            },
                            {
                              "x": 1412,
                              "y": 240
                            },
                            {
                              "x": 1387,
                              "y": 240
                            }
                          ]
                        },
                        "confidence": 0.98984003,
                        "property": {
                          "detectedBreak": {
                            "type": "LINE_BREAK"
                          }
                        },
                        "text": "7"
                      }
                    ]
                  }
                ]
              }
            ]
          }
        ],
        "confidence": 0.77101094,
        "height": 806,
        "width": 1458
      }
    ],
    "text": "15\n***************\n17"
  },
  "imagePropertiesAnnotation": {
    "cropHints": [
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 510
            },
            {
              "x": 1155
            },
            {
              "x": 1155,
              "y": 805
            },
            {
              "x": 510,
              "y": 805
            }
          ]
        },
        "confidence": 0.5625001,
        "importanceFraction": 0.67390424
      },
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 291
            },
            {
              "x": 1097
            },
            {
              "x": 1097,
              "y": 805
            },
            {
              "x": 291,
              "y": 805
            }
          ]
        },
        "confidence": 0.5625,
        "importanceFraction": 0.84238034
      },
      {
        "boundingPoly": {
          "vertices": [
            {
              "x": 218
            },
            {
              "x": 1185
            },
            {
              "x": 1185,
              "y": 805
            },
            {
              "x": 218,
              "y": 805
            }
          ]
        },
        "confidence": 0.55645883,
        "importanceFraction": 1
      }
    ],
    "dominantColors": {
      "colors": [
        {
          "color": {
            "blue": 245,
            "green": 245,
            "red": 245
          },
          "hex": "F5F5F5",
          "percent": 28.497889999999998,
          "percentRounded": 28,
          "pixelFraction": 0.8969478,
          "rgb": "245, 245,\n 245",
          "score": 0.2849789
        },
        {
          "color": {
            "blue": 119,
            "green": 119,
            "red": 119
          },
          "hex": "777777",
          "percent": 18.050036,
          "percentRounded": 18,
          "pixelFraction": 0.027630523,
          "rgb": "119, 119,\n 119",
          "score": 0.18050036
        },
        {
          "color": {
            "blue": 19,
            "green": 19,
            "red": 19
          },
          "hex": "131313",
          "percent": 7.161156,
          "percentRounded": 7,
          "pixelFraction": 0.004176707,
          "rgb": "19, 19,\n 19",
          "score": 0.07161156
        },
        {
          "color": {
            "blue": 153,
            "green": 153,
            "red": 153
          },
          "hex": "999999",
          "percent": 15.615201,
          "percentRounded": 16,
          "pixelFraction": 0.02562249,
          "rgb": "153, 153,\n 153",
          "score": 0.15615201
        },
        {
          "color": {
            "blue": 87,
            "green": 87,
            "red": 87
          },
          "hex": "575757",
          "percent": 13.579108000000002,
          "percentRounded": 14,
          "pixelFraction": 0.019678716,
          "rgb": "87, 87,\n 87",
          "score": 0.13579108
        },
        {
          "color": {
            "blue": 190,
            "green": 190,
            "red": 190
          },
          "hex": "BEBEBE",
          "percent": 11.086322,
          "percentRounded": 11,
          "pixelFraction": 0.021606425,
          "rgb": "190, 190,\n 190",
          "score": 0.11086322
        },
        {
          "color": {
            "blue": 51,
            "green": 51,
            "red": 51
          },
          "hex": "333333",
          "percent": 6.010287,
          "percentRounded": 6,
          "pixelFraction": 0.0043373494,
          "rgb": "51, 51,\n 51",
          "score": 0.06010287
        }
      ]
    }
  },
  "labelAnnotations": [
    {
      "description": "Automotive lighting",
      "mid": "/m/0768fx",
      "score": 0.92270863,
      "topicality": 0.92270863
    },
    {
      "description": "Gesture",
      "mid": "/m/0244x1",
      "score": 0.85260487,
      "topicality": 0.85260487
    },
    {
      "description": "Automotive design",
      "mid": "/m/068mqj",
      "score": 0.8496454,
      "topicality": 0.8496454
    },
    {
      "description": "Font",
      "mid": "/m/03gq5hm",
      "score": 0.7689706,
      "topicality": 0.7689706
    },
    {
      "description": "Automotive exterior",
      "mid": "/m/0h8ls87",
      "score": 0.75901467,
      "topicality": 0.75901467
    },
    {
      "description": "Auto part",
      "mid": "/m/08dz3q",
      "score": 0.7510306,
      "topicality": 0.7510306
    },
    {
      "description": "Coloring book",
      "mid": "/m/05h7rm",
      "score": 0.7148846,
      "topicality": 0.7148846
    },
    {
      "description": "Drawing",
      "mid": "/m/02csf",
      "score": 0.71441007,
      "topicality": 0.71441007
    },
    {
      "description": "Circle",
      "mid": "/m/01vkl",
      "score": 0.71318007,
      "topicality": 0.71318007
    },
    {
      "description": "Elbow",
      "mid": "/m/01vm1p",
      "score": 0.71267515,
      "topicality": 0.71267515
    },
    {
      "description": "Diagram",
      "mid": "/m/02v0m2",
      "score": 0.709528,
      "topicality": 0.709528
    },
    {
      "description": "Fashion accessory",
      "mid": "/m/0463sg",
      "score": 0.67354536,
      "topicality": 0.67354536
    },
    {
      "description": "Line art",
      "mid": "/m/0919rx",
      "score": 0.65924,
      "topicality": 0.65924
    },
    {
      "description": "Illustration",
      "mid": "/m/01kr8f",
      "score": 0.63600135,
      "topicality": 0.63600135
    },
    {
      "description": "Art",
      "mid": "/m/0jjw",
      "score": 0.63068235,
      "topicality": 0.63068235
    },
    {
      "description": "Automotive wheel system",
      "mid": "/m/0h8ly30",
      "score": 0.59069836,
      "topicality": 0.59069836
    },
    {
      "description": "Symbol",
      "mid": "/m/09ddf",
      "score": 0.58919644,
      "topicality": 0.58919644
    },
    {
      "description": "Slope",
      "mid": "/m/078hm",
      "score": 0.5772801,
      "topicality": 0.5772801
    },
    {
      "description": "Light bulb",
      "mid": "/m/0h8l4fh",
      "score": 0.5681215,
      "topicality": 0.5681215
    },
    {
      "description": "Triangle",
      "mid": "/m/07jx7",
      "score": 0.56783974,
      "topicality": 0.56783974
    },
    {
      "description": "Oval",
      "mid": "/m/06g58b",
      "score": 0.56316096,
      "topicality": 0.56316096
    },
    {
      "description": "Sketch",
      "mid": "/m/07glzq",
      "score": 0.5614814,
      "topicality": 0.5614814
    },
    {
      "description": "Monochrome",
      "mid": "/m/01mwkf",
      "score": 0.5512363,
      "topicality": 0.5512363
    },
    {
      "description": "Artwork",
      "mid": "/m/0dgsmq8",
      "score": 0.5144047,
      "topicality": 0.5144047
    },
    {
      "description": "Logo",
      "mid": "/m/0dwx7",
      "score": 0.51423424,
      "topicality": 0.51423424
    }
  ],
  "localizedObjectAnnotations": [
    {
      "boundingPoly": {
        "normalizedVertices": [
          {
            "x": 0.37679967,
            "y": 0.066629104
          },
          {
            "x": 0.91358244,
            "y": 0.066629104
          },
          {
            "x": 0.91358244,
            "y": 0.9314933
          },
          {
            "x": 0.37679967,
            "y": 0.9314933
          }
        ]
      },
      "mid": "/m/0hf58v5",
      "name": "Luggage & bags",
      "score": 0.6067607
    }
  ],
  "logoAnnotations": [
    {
      "boundingPoly": {
        "vertices": [
          {
            "x": 243,
            "y": 15
          },
          {
            "x": 1415,
            "y": 15
          },
          {
            "x": 1415,
            "y": 806
          },
          {
            "x": 243,
            "y": 806
          }
        ]
      },
      "description": "Alienware",
      "mid": "/m/042kgg",
      "score": 0.6777873
    }
  ],
  "safeSearchAnnotation": {
    "adult": "VERY_UNLIKELY",
    "medical": "VERY_UNLIKELY",
    "racy": "UNLIKELY",
    "spoof": "UNLIKELY",
    "violence": "VERY_UNLIKELY"
  },
  "textAnnotations": [
    {
      "boundingPoly": {
        "vertices": [
          {
            "x": 44,
            "y": 208
          },
          {
            "x": 1412,
            "y": 208
          },
          {
            "x": 1412,
            "y": 522
          },
          {
            "x": 44,
            "y": 522
          }
        ]
      },
      "description": "15\n***************\n17",
      "locale": "und"
    },
    {
      "boundingPoly": {
        "vertices": [
          {
            "x": 44,
            "y": 493
          },
          {
            "x": 86,
            "y": 493
          },
          {
            "x": 86,
            "y": 522
          },
          {
            "x": 44,
            "y": 522
          }
        ]
      },
      "description": "15"
    },
    {
      "boundingPoly": {
        "vertices": [
          {
            "x": 475,
            "y": 504
          },
          {
            "x": 102,
            "y": 504
          },
          {
            "x": 102,
            "y": 492
          },
          {
            "x": 475,
            "y": 492
          }
        ]
      },
      "description": "***************"
    },
    {
      "boundingPoly": {
        "vertices": [
          {
            "x": 1365,
            "y": 208
          },
          {
            "x": 1412,
            "y": 208
          },
          {
            "x": 1412,
            "y": 241
          },
          {
            "x": 1365,
            "y": 241
          }
        ]
      },
      "description": "17"
    }
  ]
}

# Image captioning

In [53]:
import base64
import os
import requests
from PIL import Image
from openai import OpenAI
from dotenv import load_dotenv

# instruction for api key
# create .env file in the project root directory e.g. /Users/hayley/Documents/p4ds/patent_search and 
# write down OPENAI_API_KEY=abcdefg where abcdefg is your api key

os.chdir('/Users/hayley/Documents/p4ds/patent_search')
load_dotenv() 
client = OpenAI()


In [54]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [122]:
# Path to your image
# image_path = "/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020050097605/p4-40.png"
# image_path = "/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020120156759/p0-6.png"
image_path = "/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p0-6.png"

# Getting the base64 string
base64_image = encode_image(image_path)

In [127]:
prompt = "Describe the image which is a drawing from a patent. \
          In your description, include \
            1. explanation of what the image looks like in one or two sentences.\
            2. explanation of parts pointed at or labeled by certain numbers or strings.\
            3. a list of all numbers and strings in the image\
              Below is an example output you can refer to:\
                ```\
              The illustration depicts <description>.\
\
Explanation of parts pointed at or labeled by certain numbers or strings:\
1. Number or letter <number or string> might indicate/ is pointing to/ appears to be associated with / is near <description>.\
2. Number or letter <number or string> might indicate/ is pointing to/ appears to be associated with / is near <description>.\
3. Number or letter <number or string> might indicate/ is pointing to/ appears to be associated with / is near <description>.\
...\
\
{'parts': [<number or string 1>, <number or string 2>, ...]}\
              ```"
        #   Omit statements that describe how the image is typical of a patent illustration, e.g.\
        #     The drawing is schematic and lacks detail, which is typical for patent illustrations aimed at highlighting the functional aspects of a design without unnecessary detail."

In [134]:
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {os.environ.get("OPENAI_API_KEY")}"
}

payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"{prompt}"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
            "detail": "low" ## low will resize image to 512 x 512 px. high will follow some complicated resizing + divide into 512 x 512 px tokens.
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

In [135]:
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-8LNLONm2R7wPzbCSWQ8BtnGzx8NjF', 'object': 'chat.completion', 'created': 1700105314, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 296, 'completion_tokens': 219, 'total_tokens': 515}, 'choices': [{'message': {'role': 'assistant', 'content': "The illustration depicts a perspective view of a cap-like object with various elements labeled.\n\nExplanation of parts pointed at or labeled by certain numbers or strings:\n1. Number 1 is pointing to the upper front part of the cap, which appears to be an area where a logo or design could be displayed.\n2. Number 2 points to what seems to be the brim or visor of the cap.\n3. Number 3 is located near the lower rear of the cap, potentially indicating an adjustable strap or fastening mechanism.\n4. Number 4 appears along the side panel of the cap, near the top edge.\n5. Number 10 is pointing to a series of small holes or perforations likely for ventilation.\n6. Number 11 might be indicating a specific feature or mat

In [136]:
print(response.json()['choices'][0]['message']['content']) # 11, 16, 19

# 참고사항 image_path = "/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020050097605/p4-40.png"
# 결과

# 1. The image is a technical drawing, most likely from a patent document showing a partial cross-sectional view of an object or system with various components. The view appears to be a side elevation showing a structure that could be a container, a vessel, or a component of a machine with a distinct sloping upper section.

# 2. Parts of the image are marked with reference numbers:

# - Number 16 appears to point to the top edge or border of the sloped section, which could signify a lid, an opening, or a part of the structure that has some specific significance like sealing or connecting.
# - Number 11 is located near the bottom edge of the image, highlighting what seems to be a series of repeating elements that may serve a particular function such as support, modular connections, or points of interaction with other parts.
# - Number 19 denotes an area beneath the sloped structure which could represent a bottom layer, base, or perhaps a mechanism or material located at that section of the drawing.

# 3. Here is a list of all the numbers in the image:

# {'numbers': [16, 11, 19]}

# --------------

# 참고사항 image_path = "/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020120156759/p0-6.png"

# 결과
# The image is a black and white line drawing of a jacket from a patent document. The illustration depicts a rear view of the jacket highlighting specific elements which are numbered for reference.

# Explanation of parts pointed at or labeled by certain numbers or strings:
# 1. Number 100 might indicate the overall garment, in this case, the jacket.
# 2. Number 200 is pointing to a rectangular area on the back of the jacket, which could represent a pocket, a panel, or a particular feature like a breathable fabric section.
# 3. Number 300 appears to be associated with a component on the sleeve, possibly a strap or an adjustment mechanism like a buckle or a fastener.
# 4. The letter 'W' is near a dashed line that may represent a fold, seam, or an area of the jacket that has some significance, such as a ventilation feature or an expansion gusset.

# {'parts': [100, 200, 300, 'W']}

# -------------

# 참고사항 image_path = "/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p0-6.png"

# 결과 The illustration depicts a three-dimensional view of what seems to be a wrist-worn device, likely a smartwatch, with various parts indicated by numbers.

# Explanation of parts pointed at or labeled by certain numbers or strings:
# 1. Number "1" might indicate the front face or display area of the device.
# 2. Number "2" might appear to be associated with a button or control interface on the side of the device.
# 3. Number "3" might indicate the bottom or underside of the device.
# 4. Number "4" might point to the contour lines suggesting the strap or band wrapping around a wrist.
# 5. Number "10" seems to be pointing to an inner component or sensor located beneath the front face of the device.
# 6. Number "11" appears to be associated with another component or feature inside the watch face area.
# 7. Number "17" could be indicating the rear cover or a sensor area on the bottom of the device.

# ```json
# {'parts': ['1', '2', '3', '4', '10', '11', '17']}
# ```


The illustration depicts a perspective view of a cap-like object with various elements labeled.

Explanation of parts pointed at or labeled by certain numbers or strings:
1. Number 1 is pointing to the upper front part of the cap, which appears to be an area where a logo or design could be displayed.
2. Number 2 points to what seems to be the brim or visor of the cap.
3. Number 3 is located near the lower rear of the cap, potentially indicating an adjustable strap or fastening mechanism.
4. Number 4 appears along the side panel of the cap, near the top edge.
5. Number 10 is pointing to a series of small holes or perforations likely for ventilation.
6. Number 11 might be indicating a specific feature or material on the inside edge of the brim.
7. Number 17 is near what could be an opening, flap, or additional feature on the back of the cap.

{
  'parts': ['1', '2', '3', '4', '10', '11', '17']
}


In [ ]:
# ## detail : low vs high 고민하면서.. size 가 low로 하면 무조건 512 x 512 라고 해서 고민임

# im = Image.open("/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020050097605/p4-41.png")
# width, height = im.size
# print(width, height) # 540 360

# im_resized = im.resize((512, 512)) # low resolution으로 하면 512,512로 무조건 resize해서 처리한다! 그래도 잘하려나? 근데 비용 때문에 그렇게 하긴 해야할거같은데 ㅠㅠ
# im_resized.show() ## 이상하게 찌그러지는데 그래도 인식 잘하려나? -> 테스트 해보니까 잘함 ㅇㅇ 


# 참고로 아래 이미지를 detail: high 로 해보면 오히려 결과가 안좋다.
# image_path = "/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p0-6.png"


## detail : high로 하면 오히려 hallucinate 함 (helmet 이라고 이야기)
# The illustration depicts a perspective view of a helmet with parts numbered for identification.

# Explanation of parts pointed at or labeled by certain numbers or strings:
# 1. Number 1 is pointing to the front edge or brim of the helmet.
# 2. Number 2 is pointing to the dashed sight line or vision path from the helmet.
# 3. Number 3 points to another dashed sight line or vision path, possibly indicating the range of peripheral vision.
# 4. Number 4 points to yet another dashed sight line or indicates the upper boundary of vision range.
# 5. Number 10 is indicating a feature inside the top part of the helmet, possibly a vent or adjustment mechanism.
# 6. Number 11 points to the interior of the helmet, likely signifying interior padding or an adjustment system.
# 7. Number 17 is placed near the bottom edge of the helmet, possibly indicating the helmet's rim or a chin strap attachment point.

# {'parts': [1, 2, 3, 4, 10, 11, 17]}

2818 1050


# Create mock data

일단 눈으로 보고 이미지 마다 들어있는 숫자 정보를 추출했음 -> 그리고 그 정보를 mock_data.json에 저장해놨음. 나중에 captioning 하고 이렇게 저장해서 주면 좋겠음.

In [4]:
import os
from glob import glob
import json

In [31]:
os.chdir('/Users/hayley/Documents/p4ds/patent_search')

In [26]:
image_dirs = glob('data_preprocess/image/*')
patent_dicts = []

# hupto 데이터 schema 참고했음 (drawings는 없었던 컬럼인데 추가함)
# [
    # { application_number : ..,
    #   publication_number : ..,
    #   patent_number : ..,
    #   drawings : [{ drawing id : .., # 0 -> 대표도, 1, 2, .. -> 도 1, 도 2, ..
    #               path : .., # 현재는 local path 인데, 향후에 gcs uri 등이어야 할것 같다. 
    #               numbers : [1,2,a,b,...], # numbers in the drawing
    #               caption : 'blah blah??' }, ...]
    # } ...
# ]

def get_sort_key(x):
    page, image = x.split('/')[-1].strip('.png').strip('p').split('-')
    sort_key = int(page) * 1000000 + int(image) # 1000000 for just a big number
    return sort_key


for dir in image_dirs[1:2]:
    # directory를 돌아다니면서,
    patent_dict = dict( 
    # 특허 번호 따기
        application_number = dir.split('/')[-1].split('pdf_')[-1], # 출원 번호
        publication_number = '', # 공개 번호
        patent_number = '', # 등록 번호
        drawings = [],
    )
    # 이미지를 순서대로 불러와서, -> 고쳐야 함 (p{d}-{d}.png 인데 숫자 부분을 문자로 인식해서 8 보다 10이 먼저 옴)
    
    images = sorted(images, key=lambda x: get_sort_key(x))
    
    # 일단 id는 0부터 시작해서 순서대로 먹이고
    
    for i, img in enumerate(images):
        
        if i == 0: #  and img.split('/')[-1] == 'p0-6.png': -> 대표도가 없는 경우도 있나? 그렇다면 복잡한 처리 필요.
            continue # skip 대표도
        
        image_dict = dict(
            drawing_id = i,
            path = img,
            numbers = [],
            caption = []
        )
        
        # 그 안에 들어있는 숫자는 내가 눈으로 적어 넣는다.
        print(img)
        
        all_numbers = input()
        image_dict['numbers'] = list(set(all_numbers.split(',')))  # turn into a list
        
        patent_dict['drawings'].append(image_dict)
    
    patent_dicts.append(patent_dict)
    

/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p8-6.png
/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p9-36.png
/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p9-37.png
/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p9-38.png
/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p9-39.png
/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p10-44.png
/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p10-45.png
/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p10-46.png
/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p10-47.png
/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/pdf_1020177009557/p11-51.png
/Users/hayley/Documents/p4ds/patent_search/data_preprocess/image/p

In [35]:
json.dump(patent_dicts, open('data_preprocess/mock_patent_info.json', 'w'), indent=4)

##

### create text -> ref_numbers -> ref_drawing_id

text_data = pd.read_excel()

In [36]:
import pandas as pd

In [39]:
text_data = pd.read_excel('data_preprocess/data.xlsx',index_col=0)

In [47]:
print(text_data.to_json(orient="records"))

[{"id":"source_1020180014052","\uc694\uc57d":"\ub0c9\uac01  \uc81c\uc5b4  \ubaa8\ub4c8,  \uc774\ub97c  \ud3ec\ud568\ud558\ub294  \uc190\ubaa9  \ub0c9\uac01  \ubc34\ub4dc  \ubc0f  \uc6e8\uc5b4\ub7ec\ube14  \ub0c9\uac01  \uc7a5\uce58\uac00  \uac1c\uc2dc\ub41c\ub2e4.  \ubcf8  \ubc1c\uba85\uc758  \uc77c  \uce21\uba74\uc5d0\ub530\ub974\uba74, \uc2e0\uccb4\uc5d0 \ub0c9\uac10\uc744 \uc77c\uc73c\ud0a4\ub294 \ub0c9\uac01 \uc81c\uc5b4 \ubaa8\ub4c8\ub85c\uc11c, \ud761\uc5f4\ud45c\uba74\uacfc \ubc1c\uc5f4\ud45c\uba74\uc744 \uac00\uc9c0\ub294 \uc5f4\uc804\uc18c\uc790\uc640; \uc0c1\uae30 \uc5f4\uc804\uc18c\uc790\uc5d0 \uc804\uc6d0\uc744 \uacf5\uae09\ud558\uae30 \uc704\ud55c \uc804\uc6d0 \uacf5\uae09\ubd80\uc640; \uc0c1\uae30 \uc5f4\uc804\uc18c\uc790\uc758 \ubc1c\uc5f4\ud45c\uba74\uc5d0 \ubd80\ucc29\ub418\uc5b4 \uc5f4\uc744 \ubc1c\uc0b0\ud558\ub294 \ubc29\uc5f4\ubd80\uc7ac(\u653e\u71b1\u90e8\u6750)\ub97c  \ud3ec\ud568\ud558\uba70,  \uc0c1\uae30  \ubc29\uc5f4\ubd80\uc7ac\ub294,  \uc77c\uc815  \ub450\u